# [Quickstart: Use agentic retrieval in Azure AI Search](https://learn.microsoft.com/en-us/azure/search/search-get-started-agentic-retrieval?tabs=foundry-perms%2Cfoundry-endpoint&pivots=programming-language-python)

In this quickstart, you use [*agentic retrieval*](https://learn.microsoft.com/en-us/azure/search/search-agentic-retrieval-concept) to create a conversational search experience powered by documents indexed in Azure AI Search and large language models (LLMs) from Azure OpenAI in Azure AI Foundry Models.

A knowledge agent orchestrates agentic retrieval by decomposing complex queries into subqueries, running the subqueries against one or more knowledge sources, and returning results with metadata. By default, the agent outputs raw content from your sources, but this quickstart uses the answer synthesis modality for natural-language answer generation.

Although you can provide your own data, this quickstart uses [sample JSON documents](https://github.com/Azure-Samples/azure-search-sample-data/tree/main/nasa-e-book/earth-at-night-json) from NASA's Earth at Night e-book. The documents describe general science topics and images of Earth at night as observed from space.

# Configure access
Before you begin, make sure you have permissions to access content and operations. We recommend Microsoft Entra ID for authentication and role-based access for authorization. You must be an Owner or User Access Administrator to assign roles. If roles aren't feasible, use key-based authentication instead.

To configure access for this quickstart, select both Azure AI Search and Azure AI Foundry:

## Azure AI Search
Azure AI Search provides the agentic retrieval pipeline. Configure access for yourself and your search service to read and write data, interact with Azure AI Foundry, and run the pipeline. To configure access for Azure AI Search:
1. Sign in to the Azure portal and select your Azure AI Foundry resource.
2. Enable role-based access.
3. Create a system-assigned managed identity.
4. Assign the following roles to yourself:
- Search Service Contributor
- Search Index Data Contributor
- Search Index Data Reader

## Azure AI Foundry
Azure AI Foundry provides the Azure OpenAI models used for embeddings, query planning, and answer generation. Grant your search service permission to use these models. To configure access for Azure AI Foundry:
1. Sign in to the Azure portal and select your Azure AI Foundry resource.
2. Assign `Cognitive Services User` to the managed identity of your search service.

# Constants and Libraries
The following variables are loaded from credentials_my.env:
- search_endpoint
- aoai_endpoint
- aoai_embedding_model
- aoai_embedding_deployment
- aoai_gpt_model
- aoai_gpt_deployment
- index_name
- knowledge_source_name
- knowledge_agent_name
- search_api_version

In [1]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv # requires python-dotenv

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

search_endpoint = os.environ["search_endpoint"]
aoai_endpoint = os.environ["aoai_endpoint"]
aoai_embedding_model = os.environ["aoai_embedding_model"]
aoai_embedding_deployment = os.environ["aoai_embedding_deployment"]
aoai_gpt_model = os.environ["aoai_gpt_model"]
aoai_gpt_deployment = os.environ["aoai_gpt_deployment"]
index_name = os.environ["index_name"]
knowledge_source_name = os.environ["knowledge_source_name"]
knowledge_agent_name = os.environ["knowledge_agent_name"]
search_api_version = os.environ["search_api_version"]

Environment variables have been loaded ;-)


# Create a search index
In Azure AI Search, an index is a structured collection of data. Add and run a code cell with the following code to define an index named `earth-at-night`, which you previously specified using the index_name variable.

The index schema contains fields for document identification and page content, embeddings, and numbers. The schema also includes configurations for semantic ranking and vector search, which uses your `text-embedding-3-large` deployment to vectorize text and match documents based on semantic similarity.

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=aoai_embedding_deployment,
                    model_name=aoai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully.")

Index 'earth-at-night' created or updated successfully.


# Upload documents to the index
Currently, the `earth-at-night` index is empty. Add and run a code cell with the following code to populate the index with JSON documents from [NASA's Earth at Night e-book](https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json). As required by Azure AI Search, each document conforms to the fields and data types defined in the index schema.

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=search_endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}' successfully.")

Documents uploaded to index 'earth-at-night' successfully.


# Create a knowledge source
A knowledge source is a reusable reference to your source data. Add and run a code cell with the following code to define a knowledge source named `earth-knowledge-source` that targets the `earth-at-night` index.

`source_data_select` specifies which index fields are accessible for retrieval and citations. Our example includes only human-readable fields to avoid lengthy, uninterpretable embeddings in responses.

In [4]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_select="id,page_chunk,page_number",
    ),
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks, api_version=search_api_version)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-knowledge-source' created or updated successfully.


# Create a knowledge agent
To target `earth-knowledge-source` and your `gpt-5-mini` deployment at query time, you need a knowledge agent. Add and run a code cell with the following code to define a knowledge agent named `earth-knowledge-agent`, which you previously specified using the `knowledge_agent_name` variable.

`reranker_threshold` ensures semantic relevance by excluding responses with a reranker score of `2.5` or lower. Meanwhile, `modality` is set to `ANSWER_SYNTHESIS`, enabling natural-language answers that cite the retrieved documents.

In [5]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality
from azure.search.documents.indexes import SearchIndexClient

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=aoai_endpoint,
    deployment_name=aoai_gpt_deployment,
    model_name=aoai_gpt_model,
)

output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True,
)

agent = KnowledgeAgent(
    name=knowledge_agent_name,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name,
            reranker_threshold=2.5,
        )
    ],
    output_configuration=output_cfg,
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_agent(agent, api_version=search_api_version)
print(f"Knowledge agent '{knowledge_agent_name}' created or updated successfully.")

Knowledge agent 'earth-knowledge-agent' created or updated successfully.


# Set up messages
Messages are the input for the retrieval route and contain the conversation history. Each message includes a role that indicates its origin, such as `system` or `user`, and content in natural language. The LLM you use determines which roles are valid.

Add and run a code cell with the following code to create a system message, which instructs `earth-knowledge-agent` to answer questions about the Earth at night and respond with "I don't know" when answers are unavailable.

In [6]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
If you don't have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

messages

[{'role': 'system',
  'content': '\nA Q&A agent that can answer questions about the Earth at night.\nIf you don\'t have the answer, respond with "I don\'t know".\n'}]

# Run the retrieval pipeline
You're ready to run agentic retrieval. Add and run a code cell with the following code to send a two-part user query to `earth-knowledge-agent`.

Given the conversation history and retrieval parameters, the agent:

1. Analyzes the entire conversation to infer the user's information need.
2. Decomposes the compound query into focused subqueries.
3. Runs the subqueries concurrently against your knowledge source.
4. Uses semantic ranker to rerank and filter the results.
5. Synthesizes the top results into a natural-language answer.

In [7]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, SearchIndexKnowledgeSourceParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=search_endpoint, agent_name=knowledge_agent_name, credential=credential)
query_1 = """
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """

messages.append({
    "role": "user",
    "content": query_1
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

Retrieved content from 'earth-knowledge-source' successfully.


# Review the response, activity, and results
Add and run a code cell with the following code to display the response, activity, and results of the retrieval pipeline.

In [8]:
import textwrap
import json

print("Response")
print(textwrap.fill(result.response[0].content[0].text, width=120))

print("Activity")
print(json.dumps([a.as_dict() for a in result.activity], indent=2))

print("Results")
print(json.dumps([r.as_dict() for r in result.references], indent=2))

Response
No relevant content was found to explain why suburban belts display larger December brightening than urban cores.
Information was found that the Phoenix nighttime street grid is sharply visible from space because the metropolitan area
is laid out on a regular grid of city blocks and streets, and that grid is most evident at night when street lighting is
seen from low-Earth orbit [ref_id:0][ref_id:1]. Major corridors such as Grand Avenue show up as diagonal, brightly lit
thoroughfares, and the lighting of large industrial and commercial properties plus brightly lit shopping centers, strip
malls, and gas stations—especially at intersections—produce bright nodes that emphasize the grid pattern
[ref_id:0][ref_id:1]. Darker, undeveloped areas (the Phoenix Mountains), agricultural fields, and the Salt River channel
provide strong contrast that makes the illuminated street grid stand out from space [ref_id:0][ref_id:1].  No specific
information was found that explains why long stretc

# Continue the conversation
Add and run a code cell with the following code to continue the conversation with `earth-knowledge-agent`.<br/>
After you send this user query, the agent fetches relevant content from `earth-knowledge-sourc`e and appends the response to the `messages` list.

In [9]:
query_2 = "How do I find lava at night?"
messages.append({
    "role": "user",
    "content": query_2
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

Retrieved content from 'earth-knowledge-source' successfully.


# Review the new response, activity, and results
Add and run a code cell with the following code to display the new response, activity, and results of the retrieval pipeline.

In [10]:
import textwrap
import json

print("Response")
print(textwrap.fill(result.response[0].content[0].text, width=120))

print("Activity")
print(json.dumps([a.as_dict() for a in result.activity], indent=2))

print("Results")
print(json.dumps([r.as_dict() for r in result.references], indent=2))

Response
To find lava at night, use satellite night-light and thermal observations together:  - Look for bright nighttime glow in
VIIRS Day/Night Band (DNB) imagery — lava shows up as a distinct bright spot against city lights (example: Etna lava
seen by VIIRS on 16 Mar 2017) [ref_id:1][ref_id:2]. - Use thermal-infrared data (e.g., Landsat 8 TIRS and OLI) to detect
the heat signature of active vents and lava flows — thermal imagery highlights hot areas even when visible light is
limited [ref_id:0]. - Combine nightlight (VIIRS DNB) and thermal (OLI/TIRS) images for the most complete view of
nighttime volcanic activity [ref_id:0]. - Note that faint natural light sources such as moonlight, airglow, and
starlight enable the DNB to detect nighttime features, improving detection of lava glow and related clouds or plumes
[ref_id:0].  Examples: VIIRS captured Etna’s nighttime lava glow (16 Mar 2017) and Landsat 8’s OLI/TIRS mapped Etna’s
active vent and thermal lava signature (image acquired 2

# Clean up resources
When you work in your own subscription, it's a good idea to finish a project by determining whether you still need the resources you created. Resources that are left running can cost you money.

In the Azure portal, you can manage your Azure AI Search and Azure AI Foundry resources by selecting All resources or Resource groups from the left pane.

Otherwise, add and run code cells with the following code to delete the objects you created in this quickstart.

## Delete the knowledge agent

In [11]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_agent(knowledge_agent_name)
print(f"Knowledge agent '{knowledge_agent_name}' deleted successfully.")

Knowledge agent 'earth-knowledge-agent' deleted successfully.


## Delete the knowledge source

In [12]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name)
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")

Knowledge source 'earth-knowledge-source' deleted successfully.


## Delete the search index

In [13]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_index(index_name)
print(f"Index '{index_name}' deleted successfully.")

Index 'earth-at-night' deleted successfully.
